In [1]:
from fastai.tabular.all import *


In [2]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('/root/.fastai/data/adult_sample/models'),Path('/root/.fastai/data/adult_sample/adult.csv'),Path('/root/.fastai/data/adult_sample/export.pkl')]

In [3]:
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [4]:
dls = TabularDataLoaders.from_csv(
    path/'adult.csv', 
    path=path, 
    y_names="salary",
    cat_names = ['workclass', 'education', 'marital-status', 
                 'occupation', 'relationship', 'race'],
    cont_names = ['age', 'fnlwgt', 'education-num'],
    procs = [Categorify, FillMissing, Normalize])

/opt/conda/lib/python3.10/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [5]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))
to = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits)

/opt/conda/lib/python3.10/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [6]:
to.xs.iloc[:2]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
12396,5,16,6,9,5,5,1,0.029956,1.420270,-0.028974
29519,5,12,5,4,2,5,1,-0.189918,-0.434909,-0.421041


In [7]:
dls = to.dataloaders(bs=64)

In [8]:
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Some-college,Married-civ-spouse,Other-service,Wife,White,False,33.000000,226629.000771,10.0,<50k
1,Local-gov,12th,Married-civ-spouse,Adm-clerical,Wife,White,False,30.000000,48541.996806,8.0,<50k
2,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,60.000000,240520.999937,9.0,>=50k
3,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,False,28.000000,175261.999291,12.0,<50k
4,Federal-gov,Assoc-voc,Divorced,Craft-repair,Not-in-family,White,False,43.000000,92774.998295,11.0,<50k
5,State-gov,Masters,Married-civ-spouse,Prof-specialty,Husband,White,False,41.000000,170924.000758,14.0,<50k
6,Private,7th-8th,Never-married,Craft-repair,Not-in-family,White,False,17.000000,168806.999470,4.0,<50k
7,Private,Bachelors,Never-married,Tech-support,Not-in-family,White,False,37.000000,34173.001252,13.0,<50k
8,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,False,20.999999,238067.999803,9.0,<50k
9,Private,HS-grad,Never-married,Machine-op-inspct,Own-child,Black,False,31.000000,166342.999505,9.0,<50k


In [9]:
learn = tabular_learner(dls, metrics=accuracy)

In [10]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.375178,0.360848,0.834306,00:05


In [11]:
learn.show_results()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary,salary_pred
0,7.0,10.0,3.0,6.0,1.0,5.0,1.0,1.129328,-0.755306,1.147229,0.0,0.0
1,5.0,12.0,3.0,6.0,1.0,5.0,1.0,0.836162,-0.149661,-0.421041,0.0,0.0
2,5.0,12.0,3.0,11.0,1.0,5.0,1.0,-0.483084,-0.549602,-0.421041,0.0,0.0
3,5.0,8.0,3.0,5.0,1.0,5.0,1.0,0.542996,-0.644857,0.755162,1.0,1.0
4,3.0,16.0,1.0,2.0,2.0,5.0,1.0,1.275911,-0.563390,-0.028974,0.0,0.0
5,5.0,12.0,7.0,9.0,5.0,3.0,1.0,0.103248,1.117867,-0.421041,0.0,0.0
6,5.0,12.0,5.0,10.0,4.0,5.0,1.0,-1.142707,0.232209,-0.421041,0.0,0.0
7,1.0,10.0,3.0,1.0,1.0,5.0,1.0,2.008825,-0.566088,1.147229,0.0,0.0
8,5.0,10.0,5.0,2.0,2.0,5.0,1.0,-1.142707,-0.459307,1.147229,0.0,0.0


In [12]:
row, clas, probs = learn.predict(df.iloc[0])

/opt/conda/lib/python3.10/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


In [13]:
row.show()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Assoc-acdm,Married-civ-spouse,#na#,Wife,White,False,49.0,101319.999102,12.0,>=50k


In [14]:
clas, probs

(tensor(1), tensor([0.4617, 0.5383]))

In [15]:
test_df = df.copy()
test_df.drop(['salary'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)
learn.get_preds(dl=dl)

/opt/conda/lib/python3.10/site-packages/fastai/tabular/core.py:312: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)


(tensor([[0.4617, 0.5383],
         [0.5550, 0.4450],
         [0.9310, 0.0690],
         ...,
         [0.6012, 0.3988],
         [0.7798, 0.2202],
         [0.6876, 0.3124]]),
 None)

In [16]:
to.xs[:3]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
12396,5,16,6,9,5,5,1,0.029956,1.420270,-0.028974
29519,5,12,5,4,2,5,1,-0.189918,-0.434909,-0.421041
30737,5,10,5,13,2,5,1,-0.263209,-0.038976,1.147229


In [17]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()